In [17]:
import pandas as pd, numpy as np
from bayes_opt import BayesianOptimization,UtilityFunction
import torch,torchvision
import math
import pprint
import dataset
from config import args
import random
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [3]:
## Model
#get comp data
train = pd.read_json('../train.json', lines=True)
test = pd.read_json('../test.json', lines=True)
sample_sub = pd.read_csv("../sample_submission.csv")
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [4]:
train_data = dataset.RNADataset(train)
test_data = dataset.RNADataset(test)

In [10]:
train_data.df.head()

,SN_filter,deg_50C,deg_Mg_50C,deg_Mg_pH10,deg_error_50C,deg_error_Mg_50C,deg_error_Mg_pH10,deg_error_pH10,deg_pH10,id,index,predicted_loop_type,reactivity,reactivity_error,seq_length,seq_scored,sequence,signal_to_noise,structure
0,1,"[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...",id_001f94081,0,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,"[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...",107,68,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,6.894,.....((((((.......)))).)).((.....((..((((((......
1,0,"[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",id_0049f53ba,1,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,"[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...",107,68,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,0.193,.....(((((((((((((((((((((((....)))))))))).)))...
2,1,"[0.9501000000000001, 1.7974999999999999, 1.499...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...",id_006f36f57,2,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,"[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.0931, 0.13290000000000002, 0.11280000000000...",107,68,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,8.800,.....((((.((.....((((.(((.....)))..((((......)...
3,0,"[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[121286.7181, 121286.7182, 121286.7181, 121286...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",id_0082d463b,3,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,"[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...",107,68,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,0.104,......((((((((((((((((......))))))))))))))))((...
4,0,"[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...",id_0087940f4,4,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,"[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...",107,68,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,0.423,.....(((((((.((((((((((((.(((((((((....)))))))...


In [15]:
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [18]:
skf = KFold(args.n_folds, shuffle=True, random_state=seed)

In [19]:
def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

In [20]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}
train_inputs = preprocess_inputs(train)
train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))

In [21]:
def rmse(y_actual, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_actual, y_pred)
    return K.sqrt(mse)

def mcrmse(y_actual, y_pred, num_scored=len(target_cols)):
    score = 0
    for i in range(num_scored):
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / num_scored
    return score

In [23]:
train_data[0]

{'target': tensor([[0.3297, 0.7556, 2.3375, 0.3581, 0.6382],
         [1.5693, 2.9830, 3.5060, 2.9683, 3.4773],
         [1.1227, 0.2526, 0.3008, 0.2589, 0.9988],
         [0.8686, 1.3789, 1.0108, 1.4552, 1.3228],
         [0.7217, 0.6376, 0.2635, 0.7244, 0.7877],
         [0.4384, 0.3313, 0.3403, 0.4971, 0.5890],
         [0.2560, 0.6763, 0.5617, 0.3819, 0.5231],
         [0.3364, 0.7525, 0.6810, 0.9115, 1.0125],
         [0.2168, 0.2080, 0.0530, 0.1668, 0.0585],
         [0.3583, 0.8908, 0.5348, 1.0613, 0.7349],
         [0.9541, 0.9898, 0.5526, 1.1317, 1.2215],
         [1.4113, 0.8679, 0.3278, 0.7468, 1.0321],
         [1.6911, 1.7403, 1.0806, 1.9049, 0.8274],
         [1.2494, 1.3795, 0.7699, 1.1030, 1.0442],
         [1.1895, 0.7746, 0.3901, 0.7317, 0.8472],
         [0.6909, 0.7190, 0.5045, 0.7649, 0.5547],
         [0.4736, 0.7229, 0.6475, 0.7010, 0.7291],
         [0.1754, 0.4069, 0.5176, 0.4327, 0.5541],
         [0.0582, 0.2632, 0.2206, 0.2360, 0.2404],
         [0.2173, 0.3